# Setup Minedojo Environment

In [1]:
from Environments import SkyRunner
import gym

env = SkyRunner.CustomEnv()
eval_env = SkyRunner.CustomEnv()

#env = gym.make("CartPole-v1")
#eval_env = gym.make("CartPole-v1")


[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


# Begin training

In [ ]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env)

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


# Load existing model

In [ ]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

model = DQN.load("last_model_working.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0

Finished with reward 7
Finished with reward 18
Finished with reward 0
Finished with reward 0
Finished with reward 10
Finished with reward 8
Finished with reward 19
Finished with reward 0
Finished with reward 0
Finished with reward 19
Finished with reward 11
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 4
Finished with reward 0
Finished with reward 8
Finished with reward 1004
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 1017
Finished with reward 1004
Finished with reward 1017
Finished with reward 13
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 14
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 7
Finished with reward 1017
Finished with reward 0
Finished with reward 0
Finished with reward 19
Finished with reward 0
Finished with reward 14
Finished w

In [1]:
from Environments import MultithreadGym

env = MultithreadGym.MultithreadGym(env_int=2, thread_int=2)

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


starting Reloader
got <CustomEnv instance>
starting Reloader
got <CustomEnv instance>


In [2]:
env.reset()

Empty: 

In [4]:
for i in range(500):
    env.step(2)
    if i % 5 == 0:
        env.reset()
env.close()

RecursionError: maximum recursion depth exceeded